<a href="https://colab.research.google.com/github/paudelsushil/adleosus/blob/main/graded_assignment4_sus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<font color='red'> Total grade assignment 4: 46.5/50 </font>**

**<font color='blue'> Good job! </font>**

# Assignment 4 - More Semantic Segmentation

## Instructions

Be aware that the code cell for `customDataset` in this template is with complete instruction.

Please follow the [instructions](assignments-setup.qmd) for setting up, completing, and submitting your assignments.

<font color='red'>

**Important Note**:

- If you are using ChatGPT or any other AI based services to correct your writings or errors in code, you are required to provide a citation link.
- For answers to theoretical questions you should provide references for the source of your answers.

</font>

## Theoretical questions



### **Q1.**

**a)**
Can you explain the limitations or drawbacks associated with each of the following methods used to extend the receptive field for deep learning in semantic segmentation:

- Using a larger kernel size for conv layers throughout the network.

- Increasing the depth of the network by stacking more convolutional layers.

- Downsampling using pooling or convolution with a stride greater than 1.

- Using dilated convolutions.

(5 points)

**b)**
What are the proposed solutions to the drawbacks of using dilated conv layers?

(5 points)

### Answer to Q1

a)
1. The limitations on using a larger kernel size for convolutional layers throughout the network are:

    - Increased computational complexity

    - Loss of local information

    - Difficulty in learning hierarchial features

2. Limitations on Increasing the depth of the network by stacking more convolutional layers are:

    - Vanishing or exploding gradients

    - Overfitting

    - Model complexity

3. The limitations associated with Downsampling using pooling or convolution with a stride greater than 1 are:

    - Loss of spatial information

    - aliasing effects

    - reduced localization accuracy

4. The limitation associated with Using Dilated convolutions method are:

    - Grid effect

    - High Computational cost

    - Memory consumption

 (sam et al., 2023 and https://chat.openai.com/)

B)  The proposed solutions to the drawbacks of using dilated conv layers are:

- Using different dialation rates: Different dialtion rates can be used to achieve different receptive field sizes.

- Using different filter sizes: Different filter sizes can be used to capture different levels of detail.

- Using different activation functions: Different activation functions can be used to improve the performance of the dilated convolution.

 (sam et al., 2023 and https://chat.openai.com/)

**<font color='red'> 3/5 points on Q1</font>**


**<font color='blue'> check below: B)  A proposed solution to the gridding artifacts of using dilated convolutional layers is called Efficient Spatial Pyramiding (ESP) block. An ESP block arranges the layers in parallel branches and has feature maps of earlier layers that have lower dilation rates as the adjacent branch of lower layers with higher dilation rates (p.33-34). Another solution is Hybrid Dilated Convolution (HDC).  In HDC, dilated convolutions are split into groups of three, each with a different dilation rate. Gaps between non-zero values are kept smaller than the convolutional layers’ kernels. Additionally, the different groups’ dilation rates do not have a common factor (p.34). Another solution used by augmented ASPP modules is to follow layers of increasing dilation rates with layers of decreasing dilation rates in 3x3 convolutional branches. There are other solutions like Large Kernel Pyramid Pooling (LKPP) which use three Hybrid Asymmetric Dilated Convolution (HADC) blocks, and a modified version where gridding is minimized by ensuring that subsequent dilation rates are never more than their kernel size (p.35).</font>**

### **Q2.**

Compare and contrast the usage of parallel and serial arrangements of dilated convolution layers for increasing the receptive field in a deep neural network. How does the arrangement affect the size and shape of the receptive field, and what are some benefits and drawbacks of each approach?

**Hint:** There are two main ways to arrange dilated convolution layers in a network: in parallel or in series. In the parallel arrangement, multiple dilated convolutions are applied in parallel to the same input feature map, and their outputs are concatenated. In the serial arrangement, multiple dilated convolutions are applied in sequence, with the output of one layer serving as the input to the next layer. The answers to this question are available in the reading.

(5 points)


### Answer to Q2

**1. Parallel Arrangement:**

In a parallel arrangement, each dilated convolution operates independently on the input feature map, leading to a simultaneous expansion of the receptive field through multiple paths.

*Benefits:*

 - Increased Receptive Field:

Parallel arrangements allow for a more rapid expansion of the receptive field compared to serial arrements, enabling the network to capture multi-scale information efficiently.

 - Diverse Contextual Information:

Each parallel path can focus on extracting different contextual information, enhancing the network's ability to understand complex spatial relationships and object structures.

*Drawbacks:*

 - Potential Redundancy:

Running multiple dilated convolutions in parallel may introduce redundancy in feature extraction, leading to increased computational overhead without significant improvement in performance.

 - Complexity Management:

Managing the complexity of parallel paths, especially in deep networks, can be challenging and may require careful design considerations to balance computational efficiency and performance.

**2. Serial Arrangement:**

In a serial arrangement, each subsequent dilated convolution layer builds upon the expanded receptive field of the previous layer, gradually increasing the network's receptive field size.

*Benefits:*

- Hierarchical Feature Learning:

Serial arrangements promote hierarchical feature learning by sequentially incorporating contextual information from previous layers, allowing the network to capture intricate details and relationships.

 - Gradual Context Integration:

The serial flow of information enables a gradual integration of contextual cues, facilitating a more structured and coherent understanding of the input data across multiple scales.

*Drawbacks:*

 - Limited Parallel Processing:

Serial arrangements may limit the network's ability to process diverse contextual information in parallel, potentially hindering the efficient extraction of multi-scale features and global context.

 - Slower Receptive Field Growth:

Compared to parallel arrangements, serial configurations may exhibit slower growth in the receptive field size, requiring more layers to achieve comparable multi-scale feature representation.

 (sam et al., 2023 and https://chat.openai.com/)


**<font color='red'> 4.5/5 points on Q2</font>**


 **<font color='blue'>Parallel dialated convolution can capture a large receptive field by combining information from multiple branches, which is useful for detecting irregularly shaped objects in indoor scenes how ever this architecture may not be precise when there is class imbalence. cascaded arrangement expands the receptive fields more than parallel design. This approach can better capture the global context of the image.</font>**


## Continue with our pipeline implementation

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install rasterio

In [ ]:
import os
import math
from pathlib import Path
from datetime import datetime, timedelta
import tqdm
import pandas as pd
import numpy as np
import cv2
import random
import rasterio as rio

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.tensorboard import SummaryWriter

from IPython.core.debugger import set_trace
from IPython.display import Image

### Pre-process the input dataset

Example code is provided below, which will allow you to get up and running with this assignment. However, you will learn best if you use the code you developed/modified from previous assignments to do the work, as you will start to see how it all fits together.

#### Input normalization

Add your own code for input normalization, or use the existing one

In [ ]:
def min_max_normalize_image(image, dtype=np.float32):
    """
    image_path(str) : Absolute path to the image patch.
    dtype (numpy datatype) : data type of the normalized image default is
        "np.float32".
    """

    # Calculate the minimum and maximum values for each band
    min_values = np.nanmin(image, axis=(1, 2))[:, np.newaxis, np.newaxis]
    max_values = np.nanmax(image, axis=(1, 2))[:, np.newaxis, np.newaxis]

    # Normalize the image data to the range [0, 1]
    normalized_img = (image - min_values) / (max_values - min_values)

    # Return the normalized image data
    return normalized_img

#### Image augmentation functions

Add the functions of of your choice here, or use the code below

In [ ]:
def flip_image_and_label(image, label, flip_type):
    """
    Applies horizontal or vertical flip augmentation to an image patch and label

    Args:
        image (numpy array) : The input image patch as a numpy array.
        label (numpy array) : The corresponding label as a numpy array.
        flip_type (string) : Based on the direction of flip. Can be either
            'hflip' or 'vflip'.

    Returns:
        A tuple containing the flipped image patch and label as numpy arrays.
    """
    if flip_type == 'hflip':
        # Apply horizontal flip augmentation to the image patch
        flipped_image = cv2.flip(image, 1)

        # Apply horizontal flip augmentation to the label
        flipped_label = cv2.flip(label, 1)

    elif flip_type == 'vflip':
        # Apply vertical flip augmentation to the image patch
        flipped_image = cv2.flip(image, 0)

        # Apply vertical flip augmentation to the label
        flipped_label = cv2.flip(label, 0)

    else:
        raise ValueError("Flip direction must be 'horizontal' or 'vertical'.")

    # Return the flipped image patch and label as a tuple
    return flipped_image.copy(), flipped_label.copy()


def rotate_image_and_label(image, label, angle):
    """
    Applies rotation augmentation to an image patch and label.

    Args:
        image (numpy array) : The input image patch as a numpy array.
        label (numpy array) : The corresponding label as a numpy array.
        angle (lost of floats) : If the list has exactly two elements they will
            be considered the lower and upper bounds for the rotation angle
            (in degrees) respectively. If number of elements are bigger than 2,
            then one value is chosen randomly as the roatation angle.

    Returns:
        A tuple containing the rotated image patch and label as numpy arrays.
    """
    if isinstance(angle, tuple) or isinstance(angle, list):
        if len(angle) == 2:
            rotation_degree = random.uniform(angle[0], angle[1])
        elif len(angle) > 2:
            rotation_degree = random.choice(angle)
        else:
            raise ValueError("Parameter degree needs at least two elements.")
    else:
        raise ValueError(
            "Rotation bound param for augmentation must be a tuple or list."
        )

    # Define the center of the image patch
    center = tuple(np.array(label.shape)/2.0)

    # Define the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center, rotation_degree, 1.0)

    # Apply rotation augmentation to the image patch
    rotated_image = cv2.warpAffine(image, rotation_matrix, image.shape[:2],
                                   flags=cv2.INTER_LINEAR)

    # Apply rotation augmentation to the label
    rotated_label = cv2.warpAffine(label, rotation_matrix, label.shape[:2],
                                   flags=cv2.INTER_NEAREST)

    # Return the rotated image patch and label as a tuple
    return rotated_image.copy(), np.rint(rotated_label.copy())

### Dataset

For assignment 4, you are working with a dataset called "PondDataset" which consists pairs of already chipped image and labels of size: `256x256` and pixel values are already in the range of `[0, 1]`.

![Structure of the pondDataset](https://github.com/agroimpacts/adleo/blob/main/images/pond-dataset.png?raw=1 ){#fig-pond}

You can find the dataset in the shared drive, which is [here](https://drive.google.com/drive/folders/1hJKRa1tNQmglErELsIEk8hXEykJadmKh?usp=share_link). Please download the entire "PondDataset" folder and place it in a convenient locations in your own Google Drive.

## Coding Assignment Part 1



This time you are provided with a pond dataset that is already chipped into tiles of `256x256` and the image values are already in the range `[0, 1]`.

However there is no CSV file to read-in and load the files. You need to modify the `ActiveLoadingDataset` you have developed in assignment 3 and modify it to get the list of required "image" and "label" files directly from the stored directory. Instead of reading from a "csv file", you will walk through the folder structure and grab all the "tiff" files for "image" and "label" folders.

Further instruction is provided in the corresponding answer template.

(10 points)

As stated above, you can adapt the code provided below, or you can use your own loader and adapt it as needed for this assignment. In this case, you need to modify the loader so that it can read chips from a directory, rather than just reading a CSV.  


### Custom dataloader

Add the custom dataloader from previous assignment and modify to fit the requirements of assignment 4.


In [ ]:
class ActiveLoadingDataset(Dataset):
    def __init__(self, src_dir, dataset_name, usage, apply_normalization=False,
                 transform=None, **kargs):
        r"""
        src_dir (str or path): Root of resource directory.
        dataset_name (str): Name of the training/validation dataset containing
                              structured folders for image, label
        usage (str): Either 'train' or 'validation'.
        transform (list): Each element is string name of the transformation to
           be used.
        """
        self.src_dir = src_dir
        self.dataset_name = dataset_name
        self.apply_normalization = apply_normalization
        self.transform = transform

        self.usage = usage
        assert self.usage in ["train", "validation"], "Usage is not recognized."

        # Get list of Image files and sort using rglob()
        img_dir = Path(src_dir) / self.dataset_name / self.usage / "images"

        image_files = sorted([str(img_path)
        for img_path in img_dir.rglob("*.tif")])

        # Get list of Label files and sort using rglob()
        lbl_dir = Path(src_dir) / self.dataset_name / self.usage / "labels"

        label_files = sorted([str(lbl_path)
        for lbl_path in lbl_dir.rglob("*.tif")])


        self.img_chips = []
        self.lbl_chips = []


      # Load images and labels
        for img_file, lbl_file in zip(image_files, label_files):
            with rio.open(img_file) as src:
              image = src.read().transpose(1, 2, 0)
            with rio.open(lbl_file) as src:
              label = src.read(1)

            self.img_chips.append(image)
            self.lbl_chips.append(label)


        print('--------{} patches cropped--------'.format(len(self.img_chips)))


    def __getitem__(self, index):

        image_chip = self.img_chips[index]
        label_chip = self.lbl_chips[index]

        # Change the code if you are using a different augmentation
        if self.usage == "train" and self.transform:

            trans_flip_ls = [m for m in self.transform if "flip" in m]
            if random.randint(0, 1) and len(trans_flip_ls) > 1:
                trans_flip = random.sample(trans_flip_ls, 1)[0]
                image_chip, label_chip = flip_image_and_label(
                    image_chip, label_chip, trans_flip
                )

            if random.randint(0, 1) and "rotate" in self.transform:
                img_chip, lbl_chip = rotate_image_and_label(
                    image_chip, label_chip, angle=[0, 90]
                )

        # Convert numpy arrays to torch tensors.
        image_tensor = torch.from_numpy(image_chip.transpose((2, 0, 1))).float()
        label_tensor = torch.from_numpy(label_chip).long()


        return image_tensor, label_tensor

    def __len__(self):

        return len(self.img_chips)

#### Loading your data

In [ ]:
src_dir = "/content/gdrive/MyDrive/adleo"
dataset_name = "PondDataset"

transform = ["hflip", "vflip", "rotate"]

In [ ]:
train_dataset = ActiveLoadingDataset(src_dir, dataset_name, usage="train",
                                     apply_normalization=False,
                                     transform=transform)

--------359 patches cropped--------


In [ ]:
train_loader = DataLoader(train_dataset,
                          batch_size = 16,
                          shuffle = True)

In [ ]:
validation_dataset = ActiveLoadingDataset(src_dir, dataset_name,
                                          usage="validation",
                                          apply_normalization=False)

--------45 patches cropped--------


In [ ]:
val_loader = DataLoader(validation_dataset, batch_size = 1, shuffle = False)

**<font color='red'> 15/15 points on CA1</font>**

### Model

#### Model architecture
Provide the model architecture you developed for Assignment 3, or use the one provided below,
a complete Unet architecture we have added for your reference.

In [ ]:
class ConvBlock(nn.Module):
    r"""This module creates a user-defined number of conv+BN+ReLU layers.
    Args:
        in_channels (int)-- number of input features.
        out_channels (int) -- number of output features.
        kernel_size (int) -- Size of convolution kernel.
        stride (int) -- decides how jumpy kernel moves along the spatial
            dimensions.
        padding (int) -- how much the input should be padded on the borders with
            zero.
        dilation (int) -- dilation ratio for enlarging the receptive field.
        num_conv_layers (int) -- Number of conv+BN+ReLU layers in the block.
        drop_rate (float) -- dropout rate at the end of the block.
    """

    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,
                 padding=1, dilation=1, num_conv_layers=2, drop_rate=0):
        super(ConvBlock, self).__init__()

        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size,
                            stride=stride, padding=padding, dilation=dilation,
                            bias=False),
                  nn.BatchNorm2d(out_channels),
                  nn.ReLU(inplace=True), ]

        if num_conv_layers > 1:
            if drop_rate > 0:
                layers += [
                    nn.Conv2d(out_channels, out_channels,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False),
                    nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
                    nn.Dropout(drop_rate),
                ] * (num_conv_layers - 1)
            else:
                layers += [
                    nn.Conv2d(out_channels, out_channels,
                              kernel_size=kernel_size, stride=stride,
                              padding=padding, dilation=dilation, bias=False),
                    nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
                ] * (num_conv_layers - 1)

        self.block = nn.Sequential(*layers)

    def forward(self, inputs):
        outputs = self.block(inputs)
        return outputs

###########################################################################

class UpconvBlock(nn.Module):
    r"""
    Decoder layer decodes the features along the expansive path.
    Args:
        in_channels (int) -- number of input features.
        out_channels (int) -- number of output features.
        upmode (str) -- Upsampling type.
            If "fixed" then a linear upsampling with scale factor of two will be
            applied using bi-linear as interpolation method. If deconv_1 is
            chosen then a non-overlapping transposed convolution will be applied
            to upsample the feature maps. If deconv_1 is chosen then an
            overlapping transposed convolution will be applied to upsample the
            feature maps.
    """

    def __init__(self, in_channels, out_channels, upmode="deconv_1"):
        super(UpconvBlock, self).__init__()

        if upmode == "fixed":
            layers = [nn.Upsample(scale_factor=2, mode="bilinear",
                                  align_corners=True), ]
            layers += [nn.BatchNorm2d(in_channels),
                       nn.Conv2d(in_channels, out_channels, kernel_size=1,
                                 stride=1, padding=0, bias=False), ]

        elif upmode == "deconv_1":
            layers = [
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2,
                                   stride=2, padding=0, dilation=1),
            ]

        elif upmode == "deconv_2":
            layers = [
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4,
                                   stride=2, padding=1, dilation=1),
            ]

        # Dense Upscaling Convolution
        elif upmode == "DUC":
            up_factor = 2
            upsample_dim = (up_factor ** 2) * out_channels
            layers = [nn.Conv2d(in_channels, upsample_dim, kernel_size=3,
                                padding=1),
                      nn.BatchNorm2d(upsample_dim),
                      nn.ReLU(inplace=True),
                      nn.PixelShuffle(up_factor), ]

        else:
            raise ValueError("Provided upsampling mode is not recognized.")

        self.block = nn.Sequential(*layers)

    def forward(self, inputs):
        return self.block(inputs)


###########################################################################

class Unet(nn.Module):
    def __init__(self, n_classes, in_channels, filter_config=None,
                 dropout_rate=0):
        super(Unet, self).__init__()

        self.in_channels = in_channels

        if not filter_config:
            filter_config = (64, 128, 256, 512, 1024, 2048)

        assert len(filter_config) == 6

        # Contraction Path
        self.encoder_1 = ConvBlock(self.in_channels, filter_config[0],
                                   num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 64x256x256
        self.encoder_2 = ConvBlock(filter_config[0], filter_config[1],
                                   num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 128x128x128
        self.encoder_3 = ConvBlock(filter_config[1], filter_config[2],
                                   num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 256x64x64
        self.encoder_4 = ConvBlock(filter_config[2], filter_config[3],
                                   num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 512x32x32
        self.encoder_5 = ConvBlock(filter_config[3], filter_config[4],
                                   num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 1024x16x16
        self.encoder_6 = ConvBlock(filter_config[4], filter_config[5],
                                   num_conv_layers=2,
                                   drop_rate=dropout_rate)  # 2048x8x8
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Expansion Path
        self.decoder_1 = UpconvBlock(filter_config[5], filter_config[4],
                                     upmode="deconv_2")  # 1024x16x16
        self.conv1 = ConvBlock(filter_config[4] * 2, filter_config[4],
                               num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_2 = UpconvBlock(filter_config[4], filter_config[3],
                                     upmode="deconv_2")  # 512x32x32
        self.conv2 = ConvBlock(filter_config[4], filter_config[3],
                               num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_3 = UpconvBlock(filter_config[3], filter_config[2],
                                     upmode="deconv_2")  # 256x64x64
        self.conv3 = ConvBlock(filter_config[3], filter_config[2],
                               num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_4 = UpconvBlock(filter_config[2], filter_config[1],
                                     upmode="deconv_2")  # 128x128x128
        self.conv4 = ConvBlock(filter_config[2], filter_config[1],
                               num_conv_layers=2, drop_rate=dropout_rate)

        self.decoder_5 = UpconvBlock(filter_config[1], filter_config[0],
                                     upmode="deconv_2")  # 64x256x256
        self.conv5 = ConvBlock(filter_config[1], filter_config[0],
                               num_conv_layers=2, drop_rate=dropout_rate)

        self.classifier = nn.Conv2d(filter_config[0], n_classes, kernel_size=1,
                                    stride=1, padding=0)  # classNumx256x256

    def forward(self, inputs):
        # set_trace()
        e1 = self.encoder_1(inputs)  # batch size x 64 x 256 x 256
        p1 = self.pool(e1)  # batch size x 64 x 128 x 128

        e2 = self.encoder_2(p1)  # batch size x 128 x 128 x 128
        p2 = self.pool(e2)  # batch size x 128 x 64 x 64

        e3 = self.encoder_3(p2)  # batch size x 256 x 64 x 64
        p3 = self.pool(e3)  # batch size x 256 x 32 x 32

        e4 = self.encoder_4(p3)  # batch size x 512 x 32 x 32
        p4 = self.pool(e4)  # batch size x 1024 x 16 x 16

        e5 = self.encoder_5(p4)  # batch size x 1024 x 16 x 16
        p5 = self.pool(e5)  # batch size x 1024 x 8 x 8

        e6 = self.encoder_6(p5)  # batch size x 2048 x 8 x 8

        d6 = self.decoder_1(e6)  # batch size x 1024 x 16 x 16
        skip1 = torch.cat((e5, d6), dim=1)  # batch size x 2048 x 16 x 16
        d6_proper = self.conv1(skip1)  # batch size x 1024 x 16 x 16

        d5 = self.decoder_2(d6_proper)  # batch size x 512 x 32 x 32
        skip2 = torch.cat((e4, d5), dim=1)  # batch size x 1024 x 32 x 32
        d5_proper = self.conv2(skip2)  # batch size x 512 x 32 x 32

        d4 = self.decoder_3(d5_proper)  # batch size x 256 x 64 x 64
        skip3 = torch.cat((e3, d4), dim=1)  # batch size x 512 x 64 x 64
        d4_proper = self.conv3(skip3)  # batch size x 256 x 64 x 64

        d3 = self.decoder_4(d4_proper)  # batch size x 128 x 128 x 128
        skip4 = torch.cat((e2, d3), dim=1)  # batch size x 256 x 128 x 128
        d3_proper = self.conv4(skip4)  # batch size x 128 x 128 x 128

        d2 = self.decoder_5(d3_proper)  # batch size x 64 x 256 x 256
        skip5 = torch.cat((e1, d2), dim=1)  # batch size x 128 x 256 x 256
        d2_proper = self.conv5(skip5)  # batch size x 64 x 256 x 256

        d1 = self.classifier(d2_proper)  # batch size x classNum x 256 x 256

        return d1

In [ ]:
class unet_sus(nn.Module):
    def __init__(self, n_classes, in_channels, filter_config=(64, 128, 256, 512, 1024), dropout_rate=0):
        super(unet_sus, self).__init__()
        self.n_classes = n_classes
        self.in_channels = in_channels

        # Remove redundant redefinitions
        # filter_config = (64, 128, 256, 512, 1024)
        # dropout_rate = 0

        # Downsample
        # 1st Block
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, filter_config[0], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[0]),
            nn.ReLU(inplace=True)
        )

        # 2nd Block
        self.conv2 = nn.Sequential(
            nn.Conv2d(filter_config[0], filter_config[1], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[1]),
            nn.ReLU(inplace=True)
        )

        # 3rd Block
        self.conv3 = nn.Sequential(
            nn.Conv2d(filter_config[1], filter_config[2], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[2]),
            nn.ReLU(inplace=True)
        )

        # 4th Block
        self.conv4 = nn.Sequential(
            nn.Conv2d(filter_config[2], filter_config[3], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[3]),
            nn.ReLU(inplace=True)
        )

        # 5th Block
        self.conv5 = nn.Sequential(
            nn.Conv2d(filter_config[3], filter_config[4], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[4]),
            nn.ReLU(inplace=True)
        )

        # 6th Block (BottleNeck)
        self.conv6 = nn.Sequential(
            nn.Conv2d(filter_config[4], filter_config[4], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[4]),
            nn.ReLU(inplace=True)
        )

        # Upsample
        # 7th Block
        self.us1 = nn.ConvTranspose2d(filter_config[4], filter_config[4], kernel_size=2, stride=2)
        self.conv7 = nn.Sequential(
            nn.Conv2d(filter_config[4], filter_config[3], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[3]),
            nn.ReLU(inplace=True)
        )

        self.us2 = nn.ConvTranspose2d(filter_config[3], filter_config[3], kernel_size=2, stride=2)
        # 8th Block
        self.conv8 = nn.Sequential(
            nn.Conv2d(filter_config[3], filter_config[2], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[2]),
            nn.ReLU(inplace=True)
        )

        self.us3 = nn.ConvTranspose2d(filter_config[2], filter_config[2], kernel_size=2, stride=2)
        # 9th Block
        self.conv9 = nn.Sequential(
            nn.Conv2d(filter_config[2], filter_config[1], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[1]),
            nn.ReLU(inplace=True)
        )

        self.us4 = nn.ConvTranspose2d(filter_config[1], filter_config[1], kernel_size=2, stride=2)
        # 10th Block
        self.conv10 = nn.Sequential(
            nn.Conv2d(filter_config[1], filter_config[0], kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filter_config[0]),
            nn.ReLU(inplace=True)
        )

        # Correct misnamed layer
        self.us5 = nn.ConvTranspose2d(filter_config[0], filter_config[0], kernel_size=2, stride=2)

        # 11th Block
        self.conv11 = nn.Sequential(
            nn.Conv2d(filter_config[0], n_classes, kernel_size=3, stride=1, padding=1)
        )

    def forward(self, inputs):
        # Downsample
        self.ds = nn.MaxPool2d(2, stride=2)

        dslyr1 = self.conv1(inputs)
        ds1 = self.ds(dslyr1)

        dslyr2 = self.conv2(ds1)
        ds2 = self.ds(dslyr2)

        dslyr3 = self.conv3(ds2)
        ds3 = self.ds(dslyr3)

        dslyr4 = self.conv4(ds3)
        ds4 = self.ds(dslyr4)

        dslyr5 = self.conv5(ds4)
        ds5 = self.ds(dslyr5)

        # Bottleneck
        btlnklyr = self.conv6(ds5)

        # Upsample
        us1 = self.us1(btlnklyr)
        ulyr1 = self.conv7(torch.cat([us1, dslyr5], 1))

        us2 = self.us2(ulyr1)
        merge6 = torch.cat([us2, dslyr4], 1)
        ulyr2 = self.conv8(merge6)

        us3 = self.us3(ulyr2)
        merge7 = torch.cat([us3, dslyr3], 1)
        ulyr3 = self.conv9(merge7)

        us4 = self.us4(ulyr3)
        merge8 = torch.cat([us4, dslyr2], 1)
        ulyr4 = self.conv10(merge8)

        us5 = self.us5(ulyr4)
        merge9 = torch.cat([us5, dslyr1], 1)
        ulyr5 = self.conv11(merge9)
        output = self.output(ulyr5)

        return output


#### Initialize your model

In [ ]:
n_classes = 2
in_channels = 6
filter_config = (32, 64, 128, 256, 512, 1024)
dropout_rate = 0.15

In [ ]:
model = Unet(n_classes, in_channels, filter_config, dropout_rate)

In [ ]:
sus_model = unet_sus(n_classes, in_channels, filter_config, dropout_rate)

### Customized loss function

You will want to add two here, which you can copy from the main assignment notebook.

In [ ]:
class BalancedCrossEntropyLoss(nn.Module):
    '''
    Balanced cross entropy loss by weighting of inverse class ratio
    Params:
        ignore_index (int): Class index to ignore
        reduction (str): Reduction method to apply to loss, return mean over batch if 'mean',
            return sum if 'sum', return a tensor of shape [N,] if 'none'
    Returns:
        Loss tensor according to arg reduction
    '''

    def __init__(self, ignore_index=-100, reduction='mean'):
        super(BalancedCrossEntropyLoss, self).__init__()
        self.ignore_index = ignore_index
        self.reduction = reduction

    def forward(self, predict, target):
        # get class weights
        class_counts = torch.bincount(target.view(-1),
                                      minlength=predict.shape[1])
        class_weights = 1.0 / torch.sqrt(class_counts.float())

        # set weight of ignore index to 0
        if self.ignore_index >= 0 and self.ignore_index < len(class_weights):
            class_weights[self.ignore_index] = 0

        # normalize weights
        class_weights /= torch.sum(class_weights)

        # apply class weights to loss function
        loss_fn = nn.CrossEntropyLoss(weight=class_weights,
                                      ignore_index=self.ignore_index,
                                      reduction=self.reduction)

        return loss_fn(predict, target)

In [ ]:
class BinaryDiceLoss(nn.Module):
    '''
        Dice loss of binary class
        Params:
            smooth (float): A float number to smooth loss, and avoid NaN error,
              default: 1
            p (int): Denominator value: \sum{x^p} + \sum{y^p}, default: 2. Used
              to control the sensitivity of the loss.
            predict (torch.tensor): Predicted tensor of shape [N, *]
            target (torch.tensor): Target tensor of same shape with predict
        Returns:
            Loss tensor
    '''
    def __init__(self, smooth=1, p=1):
        super(BinaryDiceLoss, self).__init__()
        self.smooth = smooth
        self.p = p

    def forward(self, predict, target):

        assert predict.shape == target.shape, "predict & target don't match"
        predict = predict.contiguous().view(-1)
        target = target.contiguous().view(-1)

        num = 2 * (predict * target).sum() + self.smooth
        den = (predict.pow(self.p) + target.pow(self.p)).sum() + self.smooth
        loss = 1 - num / den

        return loss


class DiceLoss(nn.Module):
    '''
        Dice loss
        Params:
            weight (torch.tensor): Weight array of shape [num_classes,]
            ignore_index (int): Class index to ignore
            predict (torch.tensor): Predicted tensor of shape [N, C, *]
            target (torch.tensor): Target tensor either in shape [N,*] or of
              same shape with predict
            reduction (str): Reduction method.
        Returns:
            same as BinaryDiceLoss
    '''
    def __init__(self, weight=None, ignore_index=-100, smooth=1, p=1,
                 reduction='sum'):
        super(DiceLoss, self).__init__()
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.dice = BinaryDiceLoss(smooth, p)
        if weight is not None:
            self.weight = weight.cuda()
        else:
            self.weight = None

    def forward(self, predict, target):
        #set_trace()
        if predict.shape == target.shape:
            pass
        elif len(predict.shape) == 4:
            target = F.one_hot(target, num_classes=predict.shape[1])\
                .permute(0, 3, 1, 2)\
                .contiguous()
        else:
            assert 'predict shape not applicable'

        self.weight = torch.Tensor([1. / predict.shape[1]] * predict.shape[1])\
            .cuda() if self.weight is None else self.weight
        predict = F.softmax(predict, dim=1)

        if self.ignore_index >= 0:
            target = torch.where(target == self.ignore_index,
                                 torch.zeros_like(target), target)
            predict = torch.where(target == self.ignore_index,
                                  torch.zeros_like(predict), predict)

        total_loss = 0
        for i in range(predict.shape[1]):
            dice_loss = self.dice(predict[:, i], target[:, i])

            assert self.weight.shape[0] == predict.shape[1], \
                    'Expect weight shape [{}], get[{}]'\
                    .format(predict.shape[1], self.weight.shape[0])
            dice_loss *= self.weight[i]
            total_loss += dice_loss

        if self.reduction == 'mean':
            loss = total_loss / predict.shape[1]
        elif self.reduction == 'sum':
            loss = total_loss

        return loss

In [ ]:
class BalancedCrossEntropyLossOHEM(nn.Module):
    """
    Online Hard Example Mining (OHEM) Cross Entropy Loss for Semantic
    Segmentation
    Params:
        ignore_index (int): Class index to ignore
        reduction (str): Reduction method to apply to loss, return mean over
            batch if 'mean', return sum if 'sum', return a tensor of shape [N,]
            if 'none'
        ohem_ratio (float): Ratio of hard examples to use in the loss function
    Returns:
        Loss tensor according to arg reduction
    """
    def __init__(self, ignore_index=-100, reduction='mean', ohem_ratio=0.25):
        super(BalancedCrossEntropyLossOHEM, self).__init__()
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.ohem_ratio = ohem_ratio

    def forward(self, predict, target):
        # calculate pixel-wise cross entropy loss
        loss_fn = nn.CrossEntropyLoss(ignore_index=self.ignore_index,
                                      reduction='none')
        pixel_losses = loss_fn(predict, target)

        # apply online hard example mining
        num_hard = int(self.ohem_ratio * pixel_losses.numel())
        _, top_indices = pixel_losses.flatten().topk(num_hard)
        ohem_losses = pixel_losses.flatten()[top_indices]

        # apply reduction to ohem losses
        if self.reduction == 'mean':
            loss = ohem_losses.mean()
        elif self.reduction == 'sum':
            loss = ohem_losses.sum()
        else:
            loss = ohem_losses

        return loss

## Coding assignment part 2: Training the network




In the sections below you need to complete functions that you need to train and validate the network over a specified number of epochs

Complete the training process. To do that you need to complete three functions.

1- A function to perform one epoch on the training dataset.

2- A function to perform one epoch on the validation dataset.

3- A function to iterate over the user-defined number of epochs

Develop this code and train/validate the model two times, each with a different loss function.

**Note:**
Detailed information is provided in the answer template.

(15 points)



**Tip:**

Q. How do you properly use the "criterion" argument inside the "train" and "validation" functions?

A. Pass the argument to the function as a string with `()` like: "BalancedCrossEntropyLoss()". Then, when it comes to using the argument inside both the `train` and `validation` functions, use `eval()` like this:

`loss = eval(criterion)(tensor A, tensor B)`

`eval()` is a built-in Python function that allows you to evaluate a string expression as a Python code. It takes a string as an argument and evaluates the expression contained in it. The result of the evaluation is then returned.

### Complete the `train` function

Complete the function to optimize over a batch of training images and labels


In [ ]:
def train(trainData, model, optimizer, criterion, gpu=True, train_loss=[]):
    """
        Train the model using provided training dataset.
        Params:
            trainData (DataLoader object) -- Batches of image chips from PyTorch
                custom dataset (AquacultureData).
            model - Choice of segmentation model.
            optimizer - Chosen optimization algorithm to update model parameters.
            criterion - Chosen function to calculate loss over training samples.
            gpu (bool, optional) -- Decide whether to use GPU, default is True.
            train_loss (empty list, optional) -- Stores average epoch loss for
            history.
    """

    model.train()

    # Mini batch iteration
    train_epoch_loss = 0
    train_batches = len(trainData)

    device = torch.device("cuda") if gpu and torch.cuda.is_available() else torch.device("cpu")

    for img_chips, labels in trainData:

        # Add code to put image and label on the 'device'.
        img_chips = img_chips.to(device)
        labels = labels.to(device)

        # Add code to clear the 'optimizer' from existing gradients (1 line)
        optimizer.zero_grad()

        # Pass image through the model to obtain prediction (1 line)
        prediction = model(img_chips)

        # calculate loss based on 'model prediction' and label (1 line)
        loss = criterion(prediction, labels)

        # Add current loss (loss.item()) to 'train_epoch_loss' counter (1 line)
        train_epoch_loss += loss.item()

        # do the backward pass to calculate gradients with respect to the loss
        loss.backward()

        # (1 line) update model weights by invoking the proper method on
        # 'optimizer'
        optimizer.step()

    train_loss.append(train_epoch_loss / train_batches)
    print('Training loss: {:.4f}'.format(train_epoch_loss / train_batches))

### Complete the `validation` function
Besides training the network, it's important to evaluate its performance on a separate "validation dataset" to ensure that it's not overfitting to the training data. The validation process is similar to the training process, but the network is set to evaluation mode using `model.eval()` and the gradients are not computed.

Complete the function to process validation images and labels

In [ ]:
def validate(valData, model, criterion, device, val_loss=[]):
    """
        Evaluate the model on separate Landsat scenes.
        Params:
            valData (DataLoader object) -- Batches of image chips from PyTorch
                custom dataset(AquacultureData)
            model -- Choice of segmentation Model.
            criterion -- Chosen function to calculate loss over validation
                samples.
            buffer: Buffer added to the targeted grid when creating dataset.
                This allows loss to calculate at non-buffered region.
            gpu (binary,optional): Decide whether to use GPU, default is True
            valLoss (empty list): To record average loss for each epoch
    """

    model.eval()

    # mini batch iteration
    eval_epoch_loss = 0

    for img_chips, labels in valData:

        img = Variable(img_chips, requires_grad=False)
        label = Variable(labels, requires_grad=False)

        #Add code to put image and label on the 'device'.
        # one line for each.
        img_chips = img.to(device)
        labels = label.to(device)

        # Pass image through the model to obtain prediction (1 line)
        prediction = model(img_chips)

        # calculate loss based on 'model prediction' and label (1 line)
        loss = criterion(prediction, labels)

        # Add current loss (loss.item()) to 'train_epoch_loss' counter (1 line)
        eval_epoch_loss += loss.item()

    print('validation loss: {}'.format(eval_epoch_loss / len(valData)))

    if val_loss != None:
        val_loss.append(float(eval_epoch_loss / len(valData)))


### Complete the epochIterator

Complete the function that iterate over the desired number of epochs



In [ ]:
def epochIterater(trainData, valData, model, criterion, WorkingFolder,
                  initial_lr, num_epochs):
    r"""
    Epoch iteration for train and evaluation.

    Arguments:
    trainData (dataloader object): Batch grouped data to train the model.
    evalData (dataloader object): Batch grouped data to evaluate the model.
    model (pytorch.nn.module object): initialized model.
    initial_lr(float): The initial learning rate.
    num_epochs (int): User-defined number of epochs to run the model.

    """

    train_loss = []
    val_loss = []

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    if device.type == "cuda":
        print('----------GPU available----------')
        gpu = True
        model = model.to(device)
    else:
        print('----------No GPU available, using CPU instead----------')
        gpu = False
        model = model

    writer = SummaryWriter(WorkingFolder)
    optimizer = optim.Adam(model.parameters(),
                           lr=initial_lr,
                           betas=(0.9, 0.999),
                           eps=1e-08,
                           weight_decay=5e-4,
                           amsgrad=False)

    scheduler = optim.lr_scheduler.StepLR(optimizer,
                                          step_size=3,
                                          gamma=0.98)

    # Add your code here
    # you need to loop through the epochs and perform the following:
    # print the current epoch number out of the total epochs
    # (e.g. "epoch: 2/10")(1 line)
    for epoch in range(1, num_epochs + 1):
      print(f"Epoch: {epoch}/{num_epochs}")

      # start the timer (1 line)
      start_epoch = datetime.now()

      # do model fit on the training data for single epoch (1 line)
      train(trainData, model, optimizer, criterion, gpu, train_loss)

      # do model validation on the validation dataset for one epoch (1 line)
      validate(valData, model, criterion, device, val_loss)

      # take a step to update the 'scheduler'. (1 line)
      scheduler.step()

      # Print the updated learning rate.
      print(f"Updated Learning Rate: {scheduler.get_lr()[0]}")

    # use "add_scalars" method with your writer to save the train and validation
    # loss to graph
    # using tensorboard package later.
      writer.add_scalars("Loss", {"Train": train_loss[-1],
                                "Val": val_loss[-1]}, epoch)

      writer.close()

    duration_in_sec = (datetime.now() - start_epoch).seconds
    duration_format = str(timedelta(seconds=duration_in_sec))
    print("--------------- Training finished in {} ---------------"\
          .format(duration_format))

### Demonstrate the code
Run the model training and validation for a specified number of epochs (e.g. 15), and then save the results. Train / validate twice, once using your first loss function, and again using your second loss function.

#### Train/validate model 1

In [ ]:
# Train/validate 1 using distribution-based class-entropy loss function
trainData = train_loader
valData = val_loader
model = model
criterion = BalancedCrossEntropyLoss()
WorkingFolder = "/content/gdrive/MyDrive/adleo/assignment4"
initial_lr = 1e-4
num_epochs = 10
epochIterater(trainData, valData, model, criterion, WorkingFolder, initial_lr, num_epochs)

----------GPU available----------
Epoch: 1/10
Training loss: 0.6030
validation loss: 0.7046905544069079
Updated Learning Rate: 0.0001
Epoch: 2/10


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:402: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Training loss: 0.4172
validation loss: 0.6201394650671217
Updated Learning Rate: 0.0001
Epoch: 3/10
Training loss: 0.3570
validation loss: 0.5442100604375203
Updated Learning Rate: 9.604e-05
Epoch: 4/10
Training loss: 0.3302
validation loss: 0.5223104748460982
Updated Learning Rate: 9.8e-05
Epoch: 5/10
Training loss: 0.3155
validation loss: 0.4902788506613837
Updated Learning Rate: 9.8e-05
Epoch: 6/10
Training loss: 0.3119
validation loss: 0.4906127005815506
Updated Learning Rate: 9.41192e-05
Epoch: 7/10
Training loss: 0.3025
validation loss: 0.4797595477766461
Updated Learning Rate: 9.604e-05
Epoch: 8/10
Training loss: 0.2849
validation loss: 0.47246027721299066
Updated Learning Rate: 9.604e-05
Epoch: 9/10
Training loss: 0.2712
validation loss: 0.47852570248974696
Updated Learning Rate: 9.2236816e-05
Epoch: 10/10
Training loss: 0.2627
validation loss: 0.5050224443276723
Updated Learning Rate: 9.41192e-05
--------------- Training finished in 0:00:03 ---------------


Save model 1 in a directory of choice in your gdrive

In [ ]:
# Save model results 1
BCEL_model_path = "/content/gdrive/MyDrive/adleo/assignment4/BCEL_model.pth"
torch.save(model.state_dict(), BCEL_model_path)


#### Train/validate model 2

In [ ]:
# Train/validate 2 using online hard example mining based loss function
trainData = train_loader
valData = val_loader
model = model
criterion = BalancedCrossEntropyLossOHEM()
WorkingFolder = "/content/gdrive/MyDrive/adleo/assignment4"
initial_lr = 1e-4
num_epochs = 10
epochIterater(trainData, valData, model, criterion, WorkingFolder, initial_lr, num_epochs)


----------GPU available----------
Epoch: 1/10
Training loss: 0.5923
validation loss: 0.8764635026454926
Updated Learning Rate: 0.0001
Epoch: 2/10
Training loss: 0.5645
validation loss: 0.8778440806600782
Updated Learning Rate: 0.0001
Epoch: 3/10
Training loss: 0.5475
validation loss: 0.9114678819974263
Updated Learning Rate: 9.604e-05
Epoch: 4/10
Training loss: 0.5379
validation loss: 0.9050062212679121
Updated Learning Rate: 9.8e-05
Epoch: 5/10
Training loss: 0.5200
validation loss: 0.9216080947054757
Updated Learning Rate: 9.8e-05
Epoch: 6/10
Training loss: 0.5004
validation loss: 0.9634029060602188
Updated Learning Rate: 9.41192e-05
Epoch: 7/10
Training loss: 0.5192
validation loss: 0.9745307981967926
Updated Learning Rate: 9.604e-05
Epoch: 8/10
Training loss: 0.5012
validation loss: 0.9438784446981218
Updated Learning Rate: 9.604e-05
Epoch: 9/10
Training loss: 0.4873
validation loss: 0.9122659802436829
Updated Learning Rate: 9.2236816e-05
Epoch: 10/10
Training loss: 0.4810
validati

Save model results 2 in a directory of choice in your gdrive

In [ ]:
# Save model results 2
OHEM_model_path = "/content/gdrive/MyDrive/adleo/assignment4/OHEM_model.pth"
torch.save(model.state_dict(), OHEM_model_path)


**<font color='red'> 15/15 points on CA2</font>**

## Evaluation and accuracy metrics

**Note:**

If you have disconnected from the Colab session or restarted the kernel, then before doing the evaluation on the validation dataset you must initialize your model once more and load the trained weights onto your model.

In [ ]:
class Evaluator(object):
    def __init__(self, num_class):
        self.num_class = num_class
        self.confusion_matrix = np.zeros((self.num_class,)*2)

    def Pixel_Accuracy(self):
        Acc = np.diag(self.confusion_matrix).sum() / self.confusion_matrix.sum()
        return Acc

    def Pixel_Accuracy_Class(self):
        Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
        Acc = np.nanmean(Acc)
        return Acc

    def Mean_Intersection_over_Union(self):
        MIoU = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) +
                    np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))
        MIoU = np.nanmean(MIoU)
        return MIoU

    def Frequency_Weighted_Intersection_over_Union(self):
        freq = np.sum(self.confusion_matrix, axis=1) /\
            np.sum(self.confusion_matrix)
        iu = np.diag(self.confusion_matrix) / (
                    np.sum(self.confusion_matrix, axis=1) +
                    np.sum(self.confusion_matrix, axis=0) -
                    np.diag(self.confusion_matrix))

        FWIoU = (freq[freq > 0] * iu[freq > 0]).sum()
        return FWIoU

    def _generate_matrix(self, gt_image, pre_image):
        mask = (gt_image >= 0) & (gt_image < self.num_class)
        label = self.num_class * gt_image[mask].astype('int') + pre_image[mask]
        count = np.bincount(label, minlength=self.num_class**2)
        confusion_matrix = count.reshape(self.num_class, self.num_class)
        return confusion_matrix

    def add_batch(self, gt_image, pre_image):
        assert gt_image.shape == pre_image.shape
        self.confusion_matrix += self._generate_matrix(gt_image, pre_image)

    def reset(self):
        self.confusion_matrix = np.zeros((self.num_class,) * 2)

## Coding Assignment Part 3

Modify `do_accuracy_evaluation` to work with the `Evaluator` class to calculate the overal metrics for a validation dataset.


More info on the specification of the function can be found in the template


Complete the code to undertake model evaluation below. Evaluate twice: once for each model trained with a different loss function.

(10 points)


### Add the code for evaluation here

In [ ]:
def do_accuracy_evaluation(model, dataloader, num_classes, filename):
    """
  Evaluates the model's accuracy on a given dataset.

  Args:
      model (torch.nn.Module): The segmentation model to evaluate.
      dataloader (torch.utils.data.DataLoader): DataLoader providing batches of data.
      num_classes (int): Number of classes in the segmentation task.
      filename (str, optional): Filename to save results (e.g., confusion matrix).

  Returns:
      float: Overall accuracy of the model.
      torch.Tensor (optional): Confusion matrix (if filename is provided).
  """
    # Set the model to evaluation mode
    model.eval()

    evaluator = Evaluator(num_classes)


    for inputs, targets in dataloader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        evaluator.update(targets, outputs)

    overall_metrics = evaluator.compute()

    with open(filename, 'w') as f:
        f.write("Overall Evaluation Metrics:\n")
        for metric, value in overall_metrics.items():
            f.write(f"{metric}: {value}\n")

    return overall_metrics




### Evaluate model 1

In [ ]:
# Demonstrate evaluation of model 1
def evaluate_model(model, loss_function, eval_data, num_classes,
                   filename, gpu=True):
    # Set the model to evaluation mode
    model.eval()
    # Move the model to the device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    evaluator = Evaluator(num_classes)  # Create an Evaluator instance
    with torch.no_grad():
        for img_chips, labels in eval_data:
            if gpu:
                img_chips = img_chips.cuda()
                labels = labels.cuda()
            # Forward pass
            outputs = model(img_chips)
            _, predicted = torch.max(outputs, 1)
            evaluator.add_batch(labels.cpu().numpy(), predicted.cpu().numpy())
    # Calculate metrics
    pixel_accuracy = evaluator.Pixel_Accuracy()
    pixel_accuracy_class = evaluator.Pixel_Accuracy_Class()
    mean_iou = evaluator.Mean_Intersection_over_Union()
    frequency_weighted_iou = evaluator.Frequency_Weighted_Intersection_over_Union()

    # Print metrics reults
    print(f"Model 1 {loss_function.__class__.__name__}:")
    print(f"Model 1 Pixel Accuracy: {pixel_accuracy:.4f}")
    print(f"Model 1 Pixel Accuracy Class: {pixel_accuracy_class:.4f}")
    print(f"Model 1 Mean IoU: {mean_iou:.4f}")
    print(f"Frequency Weighted IoU: {frequency_weighted_iou:.4f}")

    metrics = {
        "Pixel Accuracy": pixel_accuracy,
        "Pixel Accuracy Class": pixel_accuracy_class,
        "Mean Iou": mean_iou,
        "Frequency Weighted Iou": frequency_weighted_iou
    }

    report = pd.DataFrame([metrics])
    report.to_csv(filename, index=False)
    print(f"Metrics saved to {filename}\n")



In [ ]:
loss_function = BalancedCrossEntropyLoss()


evaluate_model(model, loss_function, val_loader, num_classes=3,
               filename='model1_evaluation.csv', gpu=True)

Model 1 BalancedCrossEntropyLoss:
Model 1 Pixel Accuracy: 0.8680
Model 1 Pixel Accuracy Class: 0.7112
Model 1 Mean IoU: 0.6301
Frequency Weighted IoU: 0.7651
Metrics saved to model1_evaluation.csv



<ipython-input-28-e1cd57e727c7>:11: RuntimeWarning: invalid value encountered in divide
  Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
<ipython-input-28-e1cd57e727c7>:16: RuntimeWarning: invalid value encountered in divide
  MIoU = np.diag(self.confusion_matrix) / (
<ipython-input-28-e1cd57e727c7>:26: RuntimeWarning: invalid value encountered in divide
  iu = np.diag(self.confusion_matrix) / (


### Evaluate model 2

In [ ]:
# Demonstrate evaluation of model 2
loss_function = BalancedCrossEntropyLoss()


evaluate_model(model, loss_function, val_loader, num_classes=3,
               filename='model2_evaluation.csv', gpu=True)


Model 1 BalancedCrossEntropyLoss:
Model 1 Pixel Accuracy: 0.8680
Model 1 Pixel Accuracy Class: 0.7112
Model 1 Mean IoU: 0.6301
Frequency Weighted IoU: 0.7651
Metrics saved to model2_evaluation.csv



<ipython-input-28-e1cd57e727c7>:11: RuntimeWarning: invalid value encountered in divide
  Acc = np.diag(self.confusion_matrix) / self.confusion_matrix.sum(axis=1)
<ipython-input-28-e1cd57e727c7>:16: RuntimeWarning: invalid value encountered in divide
  MIoU = np.diag(self.confusion_matrix) / (
<ipython-input-28-e1cd57e727c7>:26: RuntimeWarning: invalid value encountered in divide
  iu = np.diag(self.confusion_matrix) / (


**<font color='red'> 9/10 points on CA3</font>**

**<font color='blue'> you did the evaluation for the same model, you needed to pass model to 2 versions to avoid this.</font>**

## OPTIONAL: Visualizing activation maps and learned kernels

Using code that extracts these from intermediate layers in the network

(5 points)

### Load a multispectral satellite image

In [ ]:
with rio.open("path/to/MSI.tif") as dataset:
    # Read the image data as a numpy array and reshape to HWC
    image = dataset.read().transpose([1, 2, 0])

# Normalize the image data to be between 0 and 1
image = do_normalization(image)

# Convert the image to a PyTorch tensor and add a batch dimension
image_tensor = torch.from_numpy(image).float().permute(2, 0, 1).unsqueeze(0)

RasterioIOError: path/to/MSI.tif: No such file or directory

### Define your CNN model and load the pre-trained weights

In [ ]:
model = unet_sus()
model.load_state_dict(torch.load('path/to/traind_model_params.pth'))

print(model)

In [ ]:
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

### Visualize the activation map for a particular layer


In [ ]:
layer_index = 2  # Choose the index of the layer to visualize (0-indexed)
layer_name = f'conv{layer_index + 1}'  # Layer name for displaying in plot title
activation = {}

In [ ]:
# Set the model to evaluation mode
model.eval()
model.conv1.register_forward_hook(get_activation(layer_name))

with torch.no_grad():
    model(image_tensor)

activation = activation.squeeze().numpy()

plt.imshow(activation[layer_index])
plt.title(f'Activation Map for {layer_name}')
plt.show()